In [1]:
import numpy as np 
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
my_formatter = "{0:.2f}" #Format to 2 decimal places.

from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


### Lot of ideas and procedures take it from MANAV SEHGAL

In [2]:
df = pd.read_csv('/kaggle/input/titanic/train.csv') #data importation as DataFrame
df_test = pd.read_csv('/kaggle/input/titanic/test.csv')
test_sub = df_test.copy()

In [3]:
df = df.drop(['Ticket', 'Cabin'], axis=1)
df_test = df_test.drop(['Ticket', 'Cabin'], axis=1)
combine = [df, df_test]

In [4]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Fare             0
Embarked         2
dtype: int64

In [5]:
mode = df.Embarked.mode()
mode

0    S
dtype: object

In [6]:
df.Embarked.unique()

array(['S', 'C', 'Q', nan], dtype=object)

In [7]:
#Filling Embarked NAN values with mode()
for dataset in combine:
    dataset['Embarked'].fillna('S',inplace=True)
df[['Embarked', 'Survived']].groupby(['Embarked'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,Embarked,Survived
0,C,0.553571
1,Q,0.389610
2,S,0.339009


In [8]:
for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

In [9]:
for dataset in combine:
    dataset.loc[ dataset['Age'] <= 10, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 10) & (dataset['Age'] <= 16), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 3
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 4
    dataset.loc[ dataset['Age'] > 64, 'Age'] = 5
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,2.0,1,0,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,3.0,1,0,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",1,2.0,0,0,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,3.0,1,0,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",0,3.0,0,0,8.0500,S


In [10]:
df.Age.unique()

array([ 2.,  3., nan,  4.,  0.,  1.,  5.])

* Filling NAN values in Age feature with a Decision Tree Classifier 

In [11]:
cond = df['Age'].isna() #Setting a variable with the condition of selection
ageX_train = df[~(cond)].drop(['Age','PassengerId','Name','Embarked','Fare'],axis=1)
ageY_train = df[~(cond)].Age
ageX_test = df[(cond)].drop(['Age','PassengerId','Name','Embarked','Fare'],axis=1)
ageX_train.shape,ageY_train.shape, ageX_test.shape

((714, 5), (714,), (177, 5))

In [12]:
decision_tree = DecisionTreeClassifier(max_depth = 8)
decision_tree.fit(ageX_train, ageY_train)
Y_pred = decision_tree.predict(ageX_test)
acc_decision_tree = round(decision_tree.score(ageX_train, ageY_train) * 100, 2)
acc_decision_tree

61.2

In [13]:
rows = df[(cond)].Age.index
a=0
for elemento in rows:
    df.Age.loc[elemento] = Y_pred[a]
    a = a + 1

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [14]:
df.Age.unique()

array([2., 3., 4., 0., 1., 5.])

* Filling NAN values test data

In [15]:
df_test.Fare.fillna(df_test.Fare.mean(),inplace=True)

In [16]:
cond = df_test['Age'].isna()
ageX_train = df_test[~(cond)].drop(['Age','PassengerId','Name','Embarked','Fare'],axis=1)
ageY_train = df_test[~(cond)].Age
ageX_test = df_test[(cond)].drop(['Age','PassengerId','Name','Embarked','Fare'],axis=1)
ageX_train.shape,ageY_train.shape, ageX_test.shape

((332, 4), (332,), (86, 4))

In [17]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(ageX_train, ageY_train)
Y_pred = decision_tree.predict(ageX_test)
acc_decision_tree = round(decision_tree.score(ageX_train, ageY_train) * 100, 2)
acc_decision_tree

68.37

In [18]:
rows = df_test[(cond)].Age.index
a=0
for elemento in rows:
    df_test.Age.loc[elemento] = Y_pred[a]
    a = a + 1

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [19]:
df.Age.unique(),df_test.Age.unique()

(array([2., 3., 4., 0., 1., 5.]), array([3., 4., 2., 1., 0., 5.]))

In [20]:
#df.describe()
#df.head()
#df.shape
#df.isna().sum()
#df_test.isna().sum()

In [21]:
df[["Sex", "Survived"]].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,Sex,Survived
1,1,0.742038
0,0,0.188908


In [22]:
df[["SibSp", "Survived"]].groupby(['SibSp'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,SibSp,Survived
1,1,0.535885
2,2,0.464286
0,0,0.345395
3,3,0.250000
4,4,0.166667
5,5,0.000000
6,8,0.000000


In [23]:
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

pd.crosstab(df['Title'], df['Sex'])

Sex,0,1
Title,,
Capt,1,0
Col,2,0
Countess,0,1
Don,1,0
Dr,6,1
Jonkheer,1,0
Lady,0,1
Major,2,0
Master,40,0


In [24]:
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
 	'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    
df[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

,Title,Survived
0,Master,0.575000
1,Miss,0.702703
2,Mr,0.156673
3,Mrs,0.793651
4,Rare,0.347826


In [25]:
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

df.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,1,0,3,"Braund, Mr. Owen Harris",0,2.0,1,0,7.2500,S,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,3.0,1,0,71.2833,C,3


In [26]:
df = df.drop(['Name', 'PassengerId'], axis=1)
df_test = df_test.drop(['Name'], axis=1)
combine = [df, df_test]
df.shape, df_test.shape

((891, 9), (418, 9))

In [27]:
for dataset in combine:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

df[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,FamilySize,Survived
3,4,0.724138
2,3,0.578431
1,2,0.552795
6,7,0.333333
0,1,0.303538
4,5,0.200000
5,6,0.136364
7,8,0.000000
8,11,0.000000


In [28]:
for dataset in combine:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1

df[['IsAlone', 'Survived']].groupby(['IsAlone'], as_index=False).mean()

,IsAlone,Survived
0,0,0.505650
1,1,0.303538


In [29]:
df = df.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
df_test = df_test.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
combine = [df, df_test]

df_test.head()

,PassengerId,Pclass,Sex,Age,Fare,Embarked,Title,IsAlone
0,892,3,0,3.0,7.8292,Q,1,1
1,893,3,1,3.0,7.0000,S,3,0
2,894,2,0,4.0,9.6875,Q,1,1
3,895,3,0,2.0,8.6625,S,1,1
4,896,3,1,2.0,12.2875,S,3,0


In [30]:
df.isna().sum(),df_test.isna().sum()

(Survived    0
 Pclass      0
 Sex         0
 Age         0
 Fare        0
 Embarked    0
 Title       0
 IsAlone     0
 dtype: int64,
 PassengerId    0
 Pclass         0
 Sex            0
 Age            0
 Fare           0
 Embarked       0
 Title          0
 IsAlone        0
 dtype: int64)

In [31]:
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

df.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,IsAlone
0,0,3,0,2.0,7.2500,0,1,0
1,1,1,1,3.0,71.2833,1,3,0
2,1,3,1,2.0,7.9250,0,2,1
3,1,1,1,3.0,53.1000,0,3,0
4,0,3,0,3.0,8.0500,0,1,1


In [32]:
combine = [df, df_test]

In [33]:
df['FareBand'] = pd.qcut(df['Fare'], 5)
df[['FareBand', 'Survived']].groupby(['FareBand'], as_index=False).mean().sort_values(by='FareBand', ascending=True)

,FareBand,Survived
0,"(-0.001, 7.854]",0.217877
1,"(7.854, 10.5]",0.201087
2,"(10.5, 21.679]",0.424419
3,"(21.679, 39.688]",0.444444
4,"(39.688, 512.329]",0.642045


In [34]:
for dataset in combine:
    dataset.loc[ dataset['Fare'] <= 7.854, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.854) & (dataset['Fare'] <= 10.5), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 10.5) & (dataset['Fare'] <= 21.679), 'Fare']   = 2
    dataset.loc[(dataset['Fare'] > 21.679) & (dataset['Fare'] <= 39.688), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 39.688, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

df = df.drop(['FareBand'], axis=1)
combine = [df, df_test]

In [35]:
df.head() # We finish the cleanning and wrangling process

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,IsAlone
0,0,3,0,2.0,0,0,1,0
1,1,1,1,3.0,3,1,3,0
2,1,3,1,2.0,1,0,2,1
3,1,1,1,3.0,3,0,3,0
4,0,3,0,3.0,1,0,1,1


In [36]:
X_train = df.drop("Survived", axis=1)
Y_train = df["Survived"]
X_test  = df_test.drop("PassengerId", axis=1).copy()
X_train.shape, Y_train.shape, X_test.shape

((891, 7), (891,), (418, 7))

* Decision Tree

In [37]:
decision_tree = DecisionTreeClassifier(max_depth = 2)
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)
acc_decision_tree

78.9

* Random Forest

In [38]:
random_forest = RandomForestClassifier(criterion="gini",max_depth=2,n_estimators=200)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
acc_random_forest

80.02

In [39]:
from sklearn.ensemble import RandomForestClassifier

y = Y_train

features = ['Pclass', 'Sex', 'Age', 'Fare', 'Embarked', 'Title', 'IsAlone']
X = pd.get_dummies(X_train[features])
X_test = pd.get_dummies(X_test[features])

model = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=2)
model.fit(X, y)
predictions = model.predict(X_test)



In [40]:
predictions.shape[0]

418

In [41]:
df_test['Predictions'] = predictions

In [42]:
output = pd.DataFrame({'PassengerId': df_test.PassengerId, 'Survived': df_test.Predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


In [43]:
for dirname, _, filenames in os.walk('/kaggle'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/src/script.ipynb
/kaggle/lib/kaggle/gcp.py
/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv
/kaggle/working/__notebook__.ipynb
/kaggle/working/submission.csv
